### The theory  

Let's start by defining the discrete-time state-space model $X_{k+1} = f(X_k)$ (where $X_k$ is a vector of the position and the translational and rotational velocities, i.e. $X_k = [P_k; V_k]$), assuming a fixed sampling time $T_s$, with the quantities of interest (position and speed)

Let the state $X_k$ contain the position $P_k = (x_k, y_k, \theta_k) $ with $x_k$, $y_k$ in mm and the orientation $\theta_k$ in rad, and the velocities $V_k = (\dot x_k, \dot y_k, \dot\theta_k)$ with the translational speeds $\dot x_k, \dot y_k$  in mm/s and the rotationnal speed $\dot\theta_k$ in rad/s. The linear discrete-time state-space model is : 

$X_{k+1}= A X_k + W_k $
   
where $W_k$ is the process noise with mean zero and covariance $Q$.
<br>
For simplicity we assume that all the process noises are independent. Hence $Q$ is diagonal.

where $ Q = \begin{bmatrix} q_x & 0 & 0 & 0 & 0 & 0\\ 0 & q_y & 0 & 0 & 0 & 0 \\ 0 & 0 & q_\theta & 0 & 0 & 0 \\ 
0 & 0 & 0 & q_\dot x & 0 & 0 \\ 0 & 0 & 0 & 0 & q_\dot y & 0 \\ 0 & 0 & 0 & 0 & 0 & q_\dot\theta \end{bmatrix} $



To implement the Kalman Filter one need to distinguish two different scenarios :

1. When the camera is online : the measurements at our disposal are the position and translational and rotational speed of the robot.
2. When the camera is offline : the measurements at our disposal are only the translational and rotational speed of the robot.


We therefore consider 2 different measurements $Y_k = HX_k + N_k$, one per scenario, where $N_k$ is the measurement noise with zero mean and covariance $R$

**1. When the camera is online  the measurements $Y^{ON}$ is given by :** <br>
  $Y_k^{ON} = H^{ON} X_k + N_k^{ON} = \begin{bmatrix} 1 & 0 & 0 & 0 & 0 & 0\\ 0 & 1 & 0 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0 & 0 & 1 \end{bmatrix}X_k+N_k^{ON}$

  where $N_k^{ON}$  is the measurement noise with zero mean and covariance $R^{ON}$. 
  <br>
  For simplicity we assume that all the measurement noises are independent.
  <br>
  Hence $R^{ON}$ is a diagonal matrix, $R^{ON} = \begin{bmatrix} r_x & 0 & 0 & 0 & 0 & 0\\ 0 & r_y & 0 & 0 & 0 & 0 \\ 0 & 0 & r_\theta & 0 & 0 & 0 \\ 
  0 & 0 & 0 & r_\dot x & 0 & 0 \\ 0 & 0 & 0 & 0 & r_\dot y & 0 \\ 0 & 0 & 0 & 0 & 0 & r_\dot\theta \end{bmatrix}$
  <br>
  with :
  <br>
  $r_x = 2 $ mm² <br>
  $r_y = 2 $ mm² <br>
  $r_\theta =  0.035 $ rad² <br>
  $r_\dot x = 1.1 $ mm²/s² <br>
  $r_\dot y = 1.1 $ mm²/s² <br>
  $r_\dot \theta = 0.00087 $ rad²/s² 

**2. When the camera is offline  the measurements $Y^{OFF}$ is given by :** <br>
  $Y_k^{OFF} = H^{OFF}X_k + N_k^{OFF} =  \begin{bmatrix} 0 & 0 & 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 0 & 1 & 0 
  \\ 0 & 0 & 0 & 0 & 0 & 1\end{bmatrix}x_k+N_k^{OFF}$

  where $N_k^{OFF}$  is the measurement noise with zero mean and covariance $R^{OFF}$. 
  <br>
  For simplicity we assume that all the measurement noises are independent.
  <br>
  Hence $R^{OFF}$ is a diagonal matrix, $R^{OFF} = \begin{bmatrix} r_\dot x & 0 & 0\\ 0 & r_\dot y & 0 \\ 0 & 0 & r_\dot\theta \end{bmatrix}$
  <br>
  with :
  <br>
  $r_\dot x = 1.1 $ mm²/s² <br>
  $r_\dot y = 1.1 $ mm²/s² <br>
  $r_\dot \theta = 0.00087 $ rad²/s² 
            

We choose to only allow Thymio to go forward or rotate on itself, consequently :  

**The linear discrete-time state-space model when Thymio is going forward is :** 

  $X_{k+1}= A^F X_k + W_k = \begin{bmatrix} 1 & 0 & 0 & T_s & 0 & 0\\ 0 & 1 & 0 & 0 & T_s & 0 \\ 0 & 0 & 1 & 0 & 0 & 0 \\ 
0 & 0 & 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0 & 0 & 1 \end{bmatrix}X_k+W_k $

  
      
      
**The linear discrete-time state-space model when Thymio is rotating on itself :**

  $X_{k+1}= A^R X_k + W_k = \begin{bmatrix} 1 & 0 & 0 & 0 & 0 & 0\\ 0 & 1 & 0 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 & 0 & T_s \\ 
0 & 0 & 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0 & 0 & 1 \end{bmatrix}X_k+W_k $

For the process noises with covariance Q, we have also : 
<br>
<br>
  $ Q = \begin{bmatrix} q_x & 0 & 0 & 0 & 0 & 0\\ 0 & q_y & 0 & 0 & 0 & 0 \\ 0 & 0 & q_\theta & 0 & 0 & 0 \\ 
  0 & 0 & 0 & q_\dot x & 0 & 0 \\ 0 & 0 & 0 & 0 & q_\dot y & 0 \\ 0 & 0 & 0 & 0 & 0 & q_\dot\theta \end{bmatrix} $
  <br>
  with :
  <br>
  
$q_x = 0.011 $ mm² <br>
$q_y =  0.011 $ mm² <br>
$q_\theta = 0.0000087 $ rad² <br>
$q_\dot x = 1.1 $ mm²/s² <br>
$q_\dot y =  1.1 $ mm²/s² <br>
$q_\dot \theta = 0.00087 $ rad²/s² <br>
  <br>